In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="9MoGKNjyxriFivoPOl2m")
project = rf.workspace("signaturedetection-im4xu").project("leaf_detection-fcyc2")
version = project.version(2)
dataset = version.download("yolov12")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to leaf_detection-2 in yolov12:: 100%|██████████| 162/162 [00:00<00:00, 7281.62it/s]


In [2]:
import os
import cv2
import shutil
import albumentations as A

# Define paths
dataset_path = "/home/sourav/workplace/leaf_disease_detection/leaf_detection-2"  # Update this with your dataset path
output_path = "/home/sourav/workplace/leaf_disease_detection/dataset/cropped_data"  # Update this with your desired output path
dataset_types = ["train", "valid", "test"]

# Define augmentation pipelines
augmentations = [
    A.Compose([
        A.HorizontalFlip(p=1.0),
        A.RandomBrightnessContrast(p=1.0),
        A.Rotate(limit=30, p=1.0),
        A.GaussNoise(p=1.0)
    ]),
    A.Compose([
        A.VerticalFlip(p=1.0),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=20, p=1.0),
        A.Blur(blur_limit=3, p=1.0)
    ]),
    A.Compose([
        A.CLAHE(p=1.0),
        A.ColorJitter(p=1.0),
        A.RandomGamma(p=1.0)
    ])
]

# Function to create class-wise directories
def create_class_dirs(base_path, class_ids):
    for dataset_type in dataset_types:
        for class_id in class_ids:
            class_dir = os.path.join(base_path, dataset_type, str(class_id))
            os.makedirs(class_dir, exist_ok=True)

# Function to extract and augment objects
def extract_objects():
    class_ids = set()
    
    for dataset_type in dataset_types:
        images_path = os.path.join(dataset_path, dataset_type, "images")
        labels_path = os.path.join(dataset_path, dataset_type, "labels")
        
        for label_file in os.listdir(labels_path):
            if not label_file.endswith(".txt"):
                continue
            
            image_file = label_file.replace(".txt", ".jpg")  # Change to .png if needed
            image_path = os.path.join(images_path, image_file)
            label_path = os.path.join(labels_path, label_file)
            
            if not os.path.exists(image_path):
                continue
            
            image = cv2.imread(image_path)
            h, w, _ = image.shape
            
            with open(label_path, "r") as file:
                lines = file.readlines()
                
                for idx, line in enumerate(lines):
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    class_ids.add(class_id)
                    x_center, y_center, box_width, box_height = map(float, parts[1:])
                    
                    # Convert YOLO format to pixel values
                    x_min = int((x_center - box_width / 2) * w)
                    y_min = int((y_center - box_height / 2) * h)
                    x_max = int((x_center + box_width / 2) * w)
                    y_max = int((y_center + box_height / 2) * h)
                    
                    cropped_object = image[y_min:y_max, x_min:x_max]
                    
                    if cropped_object.size == 0:
                        continue
                    
                    output_dir = os.path.join(output_path, dataset_type, str(class_id))
                    os.makedirs(output_dir, exist_ok=True)
                    
                    output_filename = f"{image_file.split('.')[0]}_{idx}.jpg"
                    output_filepath = os.path.join(output_dir, output_filename)
                    cv2.imwrite(output_filepath, cropped_object)
                    
                    # Apply multiple augmentations only for class_id 0
                    if class_id == 0:
                        for aug_idx, aug in enumerate(augmentations):
                            augmented = aug(image=cropped_object)
                            augmented_image = augmented["image"]
                            
                            aug_output_filename = f"{image_file.split('.')[0]}_{idx}_aug{aug_idx}.jpg"
                            aug_output_filepath = os.path.join(output_dir, aug_output_filename)
                            cv2.imwrite(aug_output_filepath, augmented_image)
                    
    print("Cropping and augmentation completed!")
    create_class_dirs(output_path, class_ids)

# Run extraction and augmentation
extract_objects()


IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.


Cropping and augmentation completed!


In [5]:
len(os.listdir('/home/sourav/workplace/leaf_disease_detection/dataset/cropped_data/train/1'))

853

## Download to train yolo model to detect healthy leaf and effected leaf 


In [7]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="9MoGKNjyxriFivoPOl2m")
project = rf.workspace("signaturedetection-im4xu").project("leaf_detection-fcyc2")
version = project.version(4)
dataset = version.download("yolov12")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to leaf_detection-4 in yolov12:: 100%|██████████| 402/402 [00:00<00:00, 18869.37it/s]


## Download to train yolo model to detect only leaf and soil
 


In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="9MoGKNjyxriFivoPOl2m")
project = rf.workspace("signaturedetection-im4xu").project("leaf_detection-fcyc2")
version = project.version(5)
dataset = version.download("yolov12")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to leaf_detection-5 in yolov12:: 100%|██████████| 402/402 [00:00<00:00, 19811.19it/s]


In [6]:
from ultralytics import YOLO
import cv2
import torch
import random

# Load the trained model
model = YOLO("/home/sourav/workplace/leaf_disease_detection/runs/detect/train/weights/best.pt")  # Replace with your checkpoint path

# Set device (GPU if available, else CPU)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Load an image for inference
image_path = "/home/sourav/workplace/leaf_disease_detection/dataset/healthy_vs_disease/test/images/frame_1_0038_jpg.rf.3dcf3a605a62e72587edeac3f404690d.jpg"  # Replace with your image path
image = cv2.imread(image_path)

# Generate random colors for each class
num_classes = len(model.names)
class_colors = {i: (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)) for i in range(num_classes)}

# Perform inference
results = model(image)

# Process results
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0].tolist()  # Bounding box coordinates
        conf = box.conf[0].item()  # Confidence score
        cls = int(box.cls[0].item())  # Class ID
        
        # Assign a color based on the class
        color = class_colors.get(cls, (0, 255, 0))
        
        # Draw bounding box on the image
        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        label = f"{model.names[cls]}: {conf:.2f}"
        cv2.putText(image, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Save or display the output
output_path = "/home/sourav/workplace/leaf_disease_detection/dataset/test_image/output.jpg"
cv2.imwrite(output_path, image)
# cv2.imshow("YOLOv12 Inference", image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

print(f"Inference completed. Output saved at {output_path}")



0: 640x640 4 borers, 12 healthys, 4.5ms
Speed: 0.9ms preprocess, 4.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Inference completed. Output saved at /home/sourav/workplace/leaf_disease_detection/dataset/test_image/output.jpg
